Serial No.: Serial number of student

GRE Scores: GRE score (out of 340)

TOEFL Scores: TOEFL score (out of 120)

University Rating: University rating (out of 5)

SOP: Strength of Statement of Purpose (out of 5)

LOR: Strength of Letter of Recommendation (out of 5)

CGPA: Undergraduate CGPA (out of 10)

Research: Research experience (either 0 or 1)

Chance of Admit: Chance of admission (target/dependent variable)

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
data = pd.read_csv("Admission_Predict.csv")
data.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,1
1,2,324,107,4,4.0,4.5,8.87,1,1
2,3,316,104,3,3.0,3.5,8.00,1,0
3,4,322,110,3,3.5,2.5,8.67,1,1
4,5,314,103,2,2.0,3.0,8.21,0,0


In [6]:
df = data.drop(columns="Serial No.")

In [7]:
df.isnull().sum()

GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [8]:
df.dtypes

GRE Score              int64
TOEFL Score            int64
University Rating      int64
SOP                  float64
LOR                  float64
CGPA                 float64
Research               int64
Chance of Admit        int64
dtype: object

In [9]:
df['Chance of Admit'].value_counts(dropna=False)

0    220
1    180
Name: Chance of Admit, dtype: int64

In [10]:
X = df.drop(columns='Chance of Admit')
y = df['Chance of Admit']

In [13]:
import statsmodels.api as sma

In [14]:
temp_x = sma.add_constant(X)

In [15]:
temp_x

,const,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,1.0,337,118,4,4.5,4.5,9.65,1
1,1.0,324,107,4,4.0,4.5,8.87,1
2,1.0,316,104,3,3.0,3.5,8.00,1
3,1.0,322,110,3,3.5,2.5,8.67,1
4,1.0,314,103,2,2.0,3.0,8.21,0
...,...,...,...,...,...,...,...,...
395,1.0,324,110,3,3.5,3.5,9.04,1
396,1.0,325,107,3,3.0,3.5,9.11,1
397,1.0,330,116,4,5.0,4.5,9.45,1
398,1.0,312,103,3,3.5,4.0,8.78,0


In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train,X_test,y_train,y_test = train_test_split(temp_x,y,test_size=0.20,random_state=1)

# Logistic Regression (Full Model)

In [27]:
logreg = sma.Logit(y_train,X_train).fit()
print(logreg.summary()) #MAXIMUM_LIKELYHOOD_ESTIMATION


Optimization terminated successfully.
         Current function value: 0.269228
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:        Chance of Admit   No. Observations:                  320
Model:                          Logit   Df Residuals:                      312
Method:                           MLE   Df Model:                            7
Date:                Mon, 04 Sep 2023   Pseudo R-squ.:                  0.6084
Time:                        10:56:22   Log-Likelihood:                -86.153
converged:                       True   LL-Null:                       -220.00
Covariance Type:            nonrobust   LLR p-value:                 4.733e-54
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               -56.2064      9.808     -5.731      0.000     -75.430     -36.983
GRE Scor

In [ ]:
1-(log-Likelihood)/(LL-Null) will give Pseudo R-Square
LLR p-value < alpha 

In [19]:
df_odds = pd.DataFrame(np.exp(logreg.params), columns= ['Odds']) 

# print the dataframe
df_odds

,Odds
const,3.889098e-25
GRE Score,1.035379e+00
TOEFL Score,1.109467e+00
University Rating,1.285971e+00
SOP,1.553118e+00
LOR,1.300654e+00
CGPA,3.274481e+01
Research,2.093510e+00


In [20]:
ypred_prob = logreg.predict(X_test)
ypred_prob[:5]

398    0.335967
125    0.117820
328    0.847480
339    0.726041
172    0.956392
dtype: float64

In [21]:
ypred = [0 if x<0.5 else 1 for x in ypred_prob]

In [23]:
ypred[0:5]

[0, 0, 1, 1, 1]

In [24]:
y_test[0:5]

398    0
125    0
328    1
339    1
172    1
Name: Chance of Admit, dtype: int64

In [26]:
print(ypred[0:10])
print(y_test[0:10])

[0, 0, 1, 1, 1, 0, 0, 0, 0, 1]
398    0
125    0
328    1
339    1
172    1
342    0
197    0
291    0
29     0
284    1
Name: Chance of Admit, dtype: int64


In [29]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [30]:
cm = confusion_matrix(y_true=y_test,y_pred=ypred)
cm

array([[40,  3],
       [ 5, 32]], dtype=int64)

In [37]:
TN = cm[0,0]
FP = cm[0,1]
FN = cm[1,0]
TP = cm[1,1]

denom = TN+FP+FN+TP
numer = TN+TP
acc = numer/denom

print(acc)

0.9


In [38]:
accuracy_score(y_true=y_test,y_pred=ypred)

0.9

In [39]:
# Recall for class 0
specifity = TN / (TN + FP)
print('specifity :',specifity)

0.9302325581395349


In [41]:
# Recall for class 1
sensitivity = TP / (FN + TP)
print('sensitivity :',sensitivity)

sensitivity : 0.8648648648648649


In [42]:
from sklearn.metrics import recall_score,classification_report

In [43]:
recall_score(y_true=y_test,y_pred=ypred)

0.8648648648648649

In [45]:
print(classification_report(y_true=y_test,y_pred=ypred))

              precision    recall  f1-score   support

           0       0.89      0.93      0.91        43
           1       0.91      0.86      0.89        37

    accuracy                           0.90        80
   macro avg       0.90      0.90      0.90        80
weighted avg       0.90      0.90      0.90        80



In [55]:
13/19


0.6842105263157895

In [46]:
#precision for class 0
precision_0 = TN/(TN+FN)
precision_0

0.8888888888888888

In [47]:
#precision for class 1
precision_1 = TP/(FP+TP)
precision_1

0.9142857142857143

In [53]:
#recall for class 1 
2*precision_1*sensitivity/(precision_1+sensitivity)

0.888888888888889

In [54]:
#recall for class 0
2*precision_0*specifity/(precision_0+specifity)

0.9090909090909092

## Log Loss

In [56]:
from sklearn.metrics import log_loss
log_loss(y_true=y_test,y_pred=ypred)

3.604365338911715